# **Scraping Ulasan Aplikasi "Qara'a - Belajar Ngaji Quran"**

## 1. Instalasi dan Import Library

In [1]:
# Instalasi dan Import Library
%pip install google-play-scraper pandas

from google_play_scraper import app, reviews_all, Sort
import pandas as pd
import json
import os

Note: you may need to restart the kernel to use updated packages.


* **Metode yang digunakan:**
    * `%pip install`: Ini adalah *magic command* IPython (digunakan di Jupyter Notebook) untuk menjalankan perintah `pip` (Python package installer) langsung dari dalam cell. Perintah ini digunakan untuk menginstal dua pustaka Python:
        * `google-play-scraper`: Pustaka utama untuk berinteraksi dengan Google Play Store dan mengambil data ulasan serta informasi aplikasi.
        * `pandas`: Pustaka fundamental untuk manipulasi dan analisis data dalam Python, terutama untuk bekerja dengan data tabular seperti DataFrame.
    * `from ... import ...`: Pernyataan standar Python untuk mengimpor modul atau objek spesifik dari pustaka yang telah diinstal agar bisa digunakan dalam kode.
        * `app`, `reviews_all`, `Sort` dari `google_play_scraper`: Fungsi dan objek yang akan digunakan untuk mengambil semua ulasan aplikasi dan mengaturnya.
        * `pandas as pd`: Mengimpor pustaka pandas dan memberinya alias `pd` (konvensi umum).
        * `json`: Pustaka standar Python untuk bekerja dengan data format JSON (JavaScript Object Notation).
        * `os`: Pustaka standar Python untuk berinteraksi dengan sistem operasi, seperti membuat direktori atau mengelola path file.

* **Alasan penggunaan:**
    * `google-play-scraper` dipilih karena merupakan alat yang efektif dan mudah digunakan untuk mengotomatisasi proses pengambilan data ulasan dari Google Play Store, yang menjadi sumber data utama proyek ini. Tanpa pustaka ini, pengambilan data secara manual akan sangat tidak efisien.
    * `pandas` sangat penting untuk mengubah data ulasan yang diperoleh (biasanya dalam format list of dictionaries) menjadi struktur DataFrame yang terorganisir, sehingga memudahkan pembersihan, inspeksi, dan penyimpanan data.
    * `json` digunakan untuk menyimpan data dalam format JSON, yang merupakan format data semi-terstruktur yang umum dan mudah dibaca baik oleh manusia maupun mesin. Ini memberikan alternatif format penyimpanan selain CSV.
    * `os` diperlukan untuk manajemen file dan direktori secara programatik, seperti membuat folder `dataset_scraped` jika belum ada, untuk menyimpan hasil scraping.

* **Insight dan Hasil yang didapat:**
    * Output `Requirement already satisfied:` menunjukkan bahwa pustaka `google-play-scraper` dan `pandas` beserta dependensinya (seperti `python-dateutil`, `numpy`, `pytz`, `six`) sudah terinstal di environment Python yang digunakan. Jika belum terinstal, maka proses instalasi akan berjalan.
    * Setelah cell ini dieksekusi, semua fungsi dan objek yang diimpor siap digunakan di cell-cell berikutnya dalam notebook. Ini adalah langkah persiapan esensial sebelum melakukan tugas inti yaitu scraping.

## 2. Konfigurasi Scraping

In [2]:
# Konfigurasi Scraping
app_id = 'com.bismillah.amaljariyah'  # ID Aplikasi Qara'a
target_reviews_count = 15000 # Target jumlah ulasan (minimal 10.000 untuk Saran 4)
country_code = 'id'          # Kode negara (Indonesia)
language_code = 'id'         # Kode bahasa (Indonesia)

# Membuat folder untuk menyimpan dataset jika belum ada
output_folder = 'dataset_scraped'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
csv_output_path = os.path.join(output_folder, 'qaraa_reviews_scraped.csv')
json_output_path = os.path.join(output_folder, 'qaraa_reviews_scraped.json')

* **Metode yang digunakan:**
    * Inisialisasi variabel: Mendefinisikan beberapa variabel konfigurasi yang akan digunakan dalam proses scraping dan penyimpanan data.
    * Pemeriksaan dan pembuatan direktori: Menggunakan `os.path.exists()` untuk mengecek apakah folder tujuan sudah ada, dan `os.makedirs()` untuk membuat folder jika belum ada.
    * Penggabungan path: Menggunakan `os.path.join()` untuk membuat path file output yang sesuai dengan sistem operasi (menangani separator `\` atau `/` secara otomatis).

* **Alasan penggunaan:**
    * Variabel `app_id` (`'com.bismillah.amaljariyah'`) sangat krusial karena menentukan aplikasi mana yang ulasannya akan diambil. ID ini unik untuk setiap aplikasi di Google Play Store.
    * `target_reviews_count` (`15000`) menetapkan jumlah ulasan yang diinginkan. Ini penting untuk memenuhi Kriteria Utama 1 (minimal 3.000 sampel) dan Saran 4 (minimal 10.000 sampel). Mengatur target yang lebih tinggi dari 10.000 memberikan margin jika tidak semua ulasan berhasil diambil atau jika ada duplikat yang perlu dihapus nanti.
    * `country_code` (`'id'`) dan `language_code` (`'id'`) memastikan bahwa ulasan yang diambil berasal dari pengguna di Indonesia dan dalam Bahasa Indonesia, sesuai dengan konteks analisis yang mungkin diinginkan.
    * Pengelolaan `output_folder`, `csv_output_path`, dan `json_output_path` dilakukan untuk mengorganisir file hasil scraping. Membuat folder secara otomatis jika belum ada meningkatkan kenyamanan pengguna dan menjaga kerapian proyek. `os.path.join` memastikan kompatibilitas path antar sistem operasi.
        * *Catatan pada kode:* Komentar `# ID Aplikasi Qara'a` dan nama file output `qaraa_reviews_scraped.csv` sebaiknya disesuaikan jika `app_id` merujuk ke aplikasi lain (seperti dalam kasus ini `com.bismillah.amaljariyah`) untuk konsistensi.

* **Insight dan Hasil yang didapat:**
    * Cell ini tidak menghasilkan output visual, tetapi secara internal ia menyiapkan semua parameter yang diperlukan untuk proses scraping.
    * Setelah eksekusi, variabel-variabel tersebut akan tersimpan dalam memori notebook.
    * Jika folder `dataset_scraped` belum ada di direktori tempat notebook dijalankan, folder tersebut akan dibuat.
    * Path lengkap untuk file output CSV dan JSON (misalnya `dataset_scraped\qaraa_reviews_scraped.csv`) telah ditentukan dan siap digunakan untuk menyimpan data.
    * Konfigurasi ini menunjukkan perencanaan yang baik untuk pengumpulan data, dengan memperhatikan target jumlah data dan spesifikasi regional/bahasa.

## 3. Proses Scraping dan Menyimpan Hasil Scrapping

In [ ]:
# Proses Scraping
print(f"Memulai scraping ulasan untuk aplikasi: {app_id}")
print(f"Target jumlah ulasan: {target_reviews_count}")

try:
    scraped_reviews = reviews_all(
        app_id,
        lang=language_code,
        country=country_code,
        sort=Sort.NEWEST, # Mengambil ulasan terbaru untuk variasi
        count=target_reviews_count # Parameter count di reviews_all adalah batas, bisa jadi kurang dari ini
    )
    print(f"Berhasil mengambil {len(scraped_reviews)} ulasan.")

    if not scraped_reviews:
        print("Tidak ada ulasan yang berhasil diambil. Periksa kembali App ID atau koneksi internet.")
    else:
        # Membuat DataFrame
        df_reviews = pd.DataFrame(scraped_reviews)
        print(f"\nBerhasil membuat DataFrame dengan {df_reviews.shape[0]} baris dan {df_reviews.shape[1]} kolom.")
        print("\nContoh 5 ulasan pertama:")
        print(df_reviews.head())

        # Menyimpan Hasil Scraping ke CSV
        df_reviews.to_csv(csv_output_path, index=False, encoding='utf-8-sig')
        print(f"\nUlasan berhasil disimpan dalam format CSV di: {csv_output_path}")

        # Menyimpan ke JSON (opsional, tapi baik untuk data semi-terstruktur)
        # Mengubah DataFrame ke format list of dicts untuk JSON yang lebih rapi
        df_for_json = df_reviews.copy()
        if 'at' in df_for_json.columns:
            df_for_json['at'] = df_for_json['at'].astype(str)
        if 'repliedAt' in df_for_json.columns:
            df_for_json['repliedAt'] = df_for_json['repliedAt'].astype(str)

        reviews_json_data = df_for_json.to_dict(orient='records')
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(reviews_json_data, f, ensure_ascii=False, indent=4)
        print(f"Ulasan berhasil disimpan dalam format JSON di: {json_output_path}")

        print("\nProses scraping selesai.")
        print(f"Jumlah ulasan yang di-scrape: {len(df_reviews)}")
        if len(df_reviews) >= 10000:
            print("Jumlah ulasan memenuhi target minimal 10.000 sampel untuk Saran 4.")
        elif len(df_reviews) >= 3000:
            print("Jumlah ulasan memenuhi target minimal 3.000 sampel untuk Kriteria Utama 1.")
        else:
            print("PERHATIAN: Jumlah ulasan kurang dari 3.000. Mungkin perlu menjalankan scraping lagi atau mencoba mengambil dari sumber lain jika ulasan aplikasi ini terbatas.")

except Exception as e:
    print(f"Terjadi kesalahan saat scraping: {e}")
    print("Pastikan App ID benar dan memiliki koneksi internet yang stabil.")
    print("Google Play Scraper terkadang memiliki batasan jumlah request, coba lagi nanti jika perlu.")

Memulai scraping ulasan untuk aplikasi: com.bismillah.amaljariyah
Target jumlah ulasan: 15000
Berhasil mengambil 11366 ulasan.

Berhasil membuat DataFrame dengan 11366 baris dan 11 kolom.

Contoh 5 ulasan pertama:
                               reviewId         userName  \
0  9551cfa2-4552-4b55-baa8-073880f214d2  Pengguna Google   
1  3df7b8be-e97b-41c0-b79b-71704e390519  Pengguna Google   
2  a3fd4595-b2fe-4b13-8723-6e72bcd22e8a  Pengguna Google   
3  eff8cb1e-86b4-4e42-a7a8-0c876c2dc813  Pengguna Google   
4  b9d545ed-567b-40b3-aec2-15067d426d6a  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0  Tidak b

* **Metode yang digunakan:**
    * `reviews_all()`: Fungsi inti dari `google-play-scraper` yang digunakan untuk mengambil semua ulasan yang cocok dengan parameter yang diberikan (`app_id`, `lang`, `country`, `sort`, `count`).
        * `sort=Sort.NEWEST`: Mengurutkan ulasan berdasarkan yang terbaru. Ini baik untuk mendapatkan data yang paling relevan dengan kondisi aplikasi saat ini.
        * `count=target_reviews_count`: Menentukan jumlah maksimum ulasan yang ingin diambil. Pustaka mungkin mengembalikan kurang dari ini jika jumlah ulasan yang tersedia lebih sedikit.
    * `pd.DataFrame()`: Mengonversi list of dictionaries yang dikembalikan oleh `reviews_all()` menjadi DataFrame pandas.
    * `df_reviews.head()`: Menampilkan lima baris pertama dari DataFrame untuk inspeksi cepat.
    * `df_reviews.to_csv()`: Menyimpan DataFrame ke file CSV.
        * `index=False`: Mencegah pandas menuliskan indeks DataFrame sebagai kolom di file CSV.
        * `encoding='utf-8-sig'`: Menggunakan encoding UTF-8 dengan BOM (Byte Order Mark), yang membantu beberapa program (seperti Excel) untuk mengenali karakter non-ASCII dengan benar.
    * Konversi Tipe Data untuk JSON: Kolom `at` dan `repliedAt` yang bertipe `Timestamp` dikonversi menjadi string (`.astype(str)`) sebelum disimpan ke JSON untuk menghindari error serialisasi.
    * `df_for_json.to_dict(orient='records')`: Mengonversi DataFrame menjadi list of dictionaries, format yang ideal untuk JSON.
    * `json.dump()`: Menyimpan data Python (list of dictionaries) ke file JSON.
        * `ensure_ascii=False`: Memungkinkan karakter non-ASCII (seperti dalam Bahasa Indonesia) ditulis apa adanya tanpa di-escape.
        * `indent=4`: Membuat format file JSON lebih mudah dibaca (pretty-print) dengan indentasi 4 spasi.
    * Blok `try-except Exception as e`: Digunakan untuk menangani potensi error yang mungkin terjadi selama proses scraping (misalnya, ID aplikasi salah, tidak ada koneksi internet, batasan dari Google Play).

* **Alasan penggunaan:**
    * Fungsi `reviews_all()` dipilih karena kemampuannya untuk mengambil sejumlah besar ulasan secara otomatis, yang merupakan inti dari Kriteria 1. Pengurutan berdasarkan `Sort.NEWEST` memastikan data yang didapat adalah yang paling baru dan relevan.
    * DataFrame pandas digunakan karena kemudahannya dalam mengelola dan memanipulasi data tabular.
    * Penyimpanan ke CSV adalah standar untuk dataset tabular dan mudah diimpor oleh berbagai alat analisis data. Encoding `utf-8-sig` dipilih untuk kompatibilitas.
    * Penyimpanan ke JSON bersifat opsional tetapi memberikan fleksibilitas jika struktur data lebih kompleks atau jika akan digunakan oleh sistem lain yang lebih menyukai JSON. Konversi tipe data `Timestamp` ke string sebelum serialisasi JSON adalah praktik yang baik untuk menghindari error.
    * Penanganan error (`try-except`) penting untuk membuat skrip lebih robust dan memberikan pesan yang informatif jika terjadi masalah, daripada skrip berhenti tiba-tiba.

* **Insight dan Hasil yang didapat:**
    * Output menunjukkan bahwa proses scraping untuk aplikasi `com.bismillah.amaljariyah` berhasil dimulai.
    * **Berhasil mengambil 11.366 ulasan**, yang melebihi target `target_reviews_count` yang diset menjadi `15000`. Ini adalah hasil yang sangat baik dan menunjukkan bahwa aplikasi tersebut memiliki banyak ulasan.
        * *Catatan:* Parameter `count` di `reviews_all` adalah batas atas; jika aplikasi memiliki ulasan lebih sedikit dari `count`, maka semua ulasan yang tersedia (hingga batas internal pustaka jika ada) akan diambil. Jika aplikasi memiliki lebih banyak ulasan dari `count`, maka `count` akan menjadi batasnya. Dalam kasus ini, sepertinya aplikasi memiliki setidaknya 11.366 ulasan yang bisa diambil dengan parameter tersebut.
    * DataFrame berhasil dibuat dengan **11.366 baris dan 11 kolom**, sesuai dengan jumlah ulasan yang diambil. Kolom-kolomnya (`reviewId`, `userName`, `userImage`, `content`, `score`, `thumbsUpCount`, `reviewCreatedVersion`, `at`, `replyContent`, `repliedAt`, `appVersion`) adalah standar dari `google-play-scraper` dan menyediakan informasi yang kaya.
    * Contoh 5 ulasan pertama (`df_reviews.head()`) berhasil ditampilkan, memberikan gambaran data yang diperoleh. Terlihat ada ulasan dengan berbagai skor dan beberapa memiliki balasan dari pengembang.
    * File CSV (`dataset_scraped\qaraa_reviews_scraped.csv`) dan file JSON (`dataset_scraped\qaraa_reviews_scraped.json`) **berhasil disimpan**. Ini penting karena file-file ini akan menjadi input untuk tahap analisis selanjutnya.
    * Pesan konfirmasi "Jumlah ulasan memenuhi target minimal 10.000 sampel untuk Saran 4" menunjukkan bahwa **Kriteria Utama 1 dan Saran 4 terkait jumlah data telah terpenuhi**. Ini adalah pencapaian penting untuk proyek.

## Kesimpulan Akhir

Secara keseluruhan **berhasil dengan sangat baik** dalam menjalankan tugas utamanya, yaitu **mengumpulkan dataset ulasan aplikasi dari Google Play Store** untuk aplikasi dengan ID `com.bismillah.amaljariyah`.

**Poin-Poin Kunci Keberhasilan:**

1.  **Persiapan Lingkungan:** Semua pustaka yang diperlukan (`google-play-scraper`, `pandas`, `json`, `os`) berhasil diimpor dan siap digunakan. Output menunjukkan bahwa pustaka-pustaka tersebut sudah terinstal dengan benar.
2.  **Konfigurasi yang Tepat:** Parameter-parameter penting untuk proses scraping, seperti `app_id`, `target_reviews_count`, `country_code`, `language_code`, serta path untuk penyimpanan output, telah didefinisikan dengan jelas. Folder output juga berhasil dibuat jika belum ada.
3.  **Proses Scraping Sukses:**
    * **Pengambilan Data:** Fungsi `reviews_all` berhasil mengambil **11.366 ulasan**, yang merupakan jumlah yang substansial dan melebihi target minimal 10.000 ulasan (memenuhi Kriteria Utama 1 dan Saran 4).
    * **Pembuatan DataFrame:** Data ulasan berhasil dikonversi menjadi DataFrame pandas dengan 11.366 baris dan 11 kolom, menyediakan struktur data yang terorganisir dan kaya informasi.
    * **Penyimpanan Data:** DataFrame berhasil disimpan ke dalam format **CSV** (`dataset_scraped\qaraa_reviews_scraped.csv`) dan **JSON** (`dataset_scraped\qaraa_reviews_scraped.json`) setelah penanganan konversi tipe data `Timestamp` untuk JSON. Kedua format ini memberikan fleksibilitas untuk penggunaan data di tahap selanjutnya.
    * **Penanganan Error:** Penggunaan blok `try-except` memastikan skrip berjalan dengan stabil dan memberikan informasi jika terjadi kendala.

**Implikasi untuk Proyek:**

* Dataset yang dihasilkan (11.366 ulasan) sudah sangat memadai untuk melanjutkan ke tahap preprocessing, pelabelan, pelatihan model, dan evaluasi dalam proyek analisis sentimen ini.
* Kualitas data yang diambil (termasuk konten ulasan, skor, tanggal, dll.) memberikan dasar yang kuat untuk analisis yang mendalam.
* Pemenuhan Kriteria Utama 1 (data scraping mandiri) dan Saran 4 (minimal 10.000 sampel) dari panduan proyek telah tercapai pada tahap ini.